In [1]:
# Set File Paths
tripdelaysFilePath = "/databricks-datasets/flights/departuredelays.csv"
airportsnaFilePath = "/databricks-datasets/flights/airport-codes-na.txt"

# Obtain airports dataset
# Note, this dataset is tab-delimited with a header
airportsna = spark.read.csv(airportsnaFilePath, header='true', inferSchema='true', sep='\t')
airportsna.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
# Note, this dataset is comma-delimited with a header
departureDelays = spark.read.csv(tripdelaysFilePath, header='true')
departureDelays.createOrReplaceTempView("departureDelays")
departureDelays.cache()

In [2]:
departureDelays.show()

In [3]:
airportsna.show()

In [4]:
# Available IATA codes from the departuredelays sample dataset
tripIATA = spark.sql("select distinct iata from (select distinct origin as iata from departureDelays union all select distinct destination as iata from departureDelays) a")
tripIATA.createOrReplaceTempView("tripIATA")
tripIATA.show()

In [5]:
# Only include airports with atleast one trip from the 
# `departureDelays` dataset
airports = spark.sql("select f.IATA, f.City, f.State, f.Country from airports_na f join tripIATA t on t.IATA = f.IATA")
airports.createOrReplaceTempView("airports")
airports.cache()

In [6]:
# Build `departureDelays_geo` DataFrame
# Obtain key attributes such as Date of flight, delays, distance, 
# and airport information (Origin, Destination)  
departureDelays_geo = spark.sql("select cast(f.date as int) as tripid, cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ''), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, cast(f.delay as int), cast(f.distance as int), f.origin as src, f.destination as dst, o.city as city_src, d.city as city_dst, o.state as state_src, d.state as state_dst from departuredelays f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# Create Temporary View and cache
departureDelays_geo.createOrReplaceTempView("departureDelays_geo")
departureDelays_geo.cache()

In [7]:
# Review the top 10 rows of the `departureDelays_geo` DataFrame
departureDelays_geo.show(10)

In [8]:
# Note, ensure you have already installed 
# the GraphFrames spark-package
from pyspark.sql.functions import *
from graphframes import *

# Create Vertices (airports) and Edges (flights)
tripVertices = airports.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays_geo.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache Vertices and Edges
tripEdges.cache()
tripVertices.cache()

In [9]:
display(tripEdges)

In [10]:
tripGraph = GraphFrame(tripVertices, tripEdges)

In [11]:
print "Airports: %d" % tripGraph.vertices.count()
print "Trips: %d" % tripGraph.edges.count()

In [12]:
tripGraph.edges.groupBy().max("delay").collect()

In [13]:
print "On-time / Early Flights: %d" % tripGraph.edges.filter("delay <= 0").count()
print "Delayed Flights: %d" % tripGraph.edges.filter("delay > 0").count()

<p>Which flights departing Seattle are most likely to have significant delay ? </p>

In [15]:
tripGraph.edges\
  .filter("src = 'SEA' and delay > 0")\
  .groupBy("src", "dst")\
  .avg("delay")\
  .sort(desc("avg(delay)"))\
  .show(5)

<p>Which states tend to have significant delays departing from Seattle</p>

In [17]:
# States with the longest cumulative delays (with individual
# delays > 100 minutes) (origin: Seattle)
display(tripGraph.edges.filter("src = 'SEA' and delay > 100"))

In [18]:
display(tripGraph.degrees.sort(desc("degree")).limit(20))

<p>Determining the top transfer airports</p>

In [20]:
inDeg = tripGraph.inDegrees #flights coming to airport
outDeg = tripGraph.outDegrees #flights leaving airport

In [21]:
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id).drop(outDeg.id).selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio").cache()
degreeRatio.show(10)

In [22]:
degreeRatio.join(airports, degreeRatio.id == airports.IATA).show(10)

In [23]:
transferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA).selectExpr("id","city","degreeRatio").filter("degreeRatio between 0.9 and 1.1")
display(transferAirports.orderBy("degreeRatio").limit(10))

In [24]:
tripGraph.bfs(fromExpr = "id = 'SEA'", toExpr = "id = 'SFO'", maxPathLength=1)